<a href="https://colab.research.google.com/github/luanfelixcoding/climate-change-impact-economy/blob/master/climate_change_impact_economy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link para base de dados: https://www.kaggle.com/datasets/tarunrm09/climate-change-indicators

In [ ]:
# conectando com o drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
file_path: str = "/content/drive/My Drive/climate_change_dataset/climate_change_indicators.csv"

# dataset
df = pd.read_csv(file_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             225 non-null    int64  
 1   Country              225 non-null    object 
 2   ISO2                 223 non-null    object 
 3   ISO3                 225 non-null    object 
 4   Indicator            225 non-null    object 
 5   Unit                 225 non-null    object 
 6   Source               225 non-null    object 
 7   CTS_Code             225 non-null    object 
 8   CTS_Name             225 non-null    object 
 9   CTS_Full_Descriptor  225 non-null    object 
 10  F1961                188 non-null    float64
 11  F1962                189 non-null    float64
 12  F1963                188 non-null    float64
 13  F1964                188 non-null    float64
 14  F1965                188 non-null    float64
 15  F1966                192 non-null    flo

In [4]:
# verificando valores nulos em cada coluna
def checking_null_values(df: pd.DataFrame) -> None:
  for col in df.columns:
    if df[col].isnull().sum() > 0:
      print(f"{df[col].name} = {df[col].isnull().sum()}")

In [5]:
# Useless code block

selected_rows = df[df['Country'] == 'Brazil']
selected_rows

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
26,27,Brazil,BR,BRA,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,0.922,1.153,1.516,1.457,1.363,1.148,1.517,1.477,1.154,0.926


In [6]:
# taking the years
years = [col for col in df.columns if col.startswith('F')]

# Derrete o dataset em variaveis e valores
# 'Country' e 'ISO3' mantemos, nomeamos uma coluna como 'Year' que recebe a variavel 'years'
df_melted = df.melt(id_vars=['Country', 'ISO3'], value_vars=years, var_name='Year', value_name='Temperature_Change')

# Na coluna 'Year' extraindo todos os numeros e convertendo para int, "(\d+)" permite extrair numeros compostos
df_melted['Year'] = df_melted['Year'].str.extract("(\d+)").astype(int)
# checking_null_values(df_melted) # output: Temperature_Change = 1490

# Removendo valores nulos; inplace=True faz as alterações no proprio df, inplace=False cria outro df com os valores removidos
df_melted.dropna(subset=['Temperature_Change'], inplace=True)
checking_null_values(df_melted) # output: Temperature_Change = 0

## Plotando Gráfico referente as mudanças climáticas durante os anos

In [10]:
# Agrupando a coluna 'Year' e pegando a media de seus valores em conjunto
global_temp_trend = df_melted.groupby('Year', as_index=False)['Temperature_Change'].mean()

fig_global = go.Figure()

fig_global.add_trace(go.Scatter(
    x=global_temp_trend["Year"],
    y=global_temp_trend["Temperature_Change"],
    mode='lines',
    line=dict(color='red', width=2),  # Linha vermelha
    name='Tendência Global'
))

fig_global.add_trace(go.Scatter(
    x=global_temp_trend["Year"],
    y=global_temp_trend["Temperature_Change"],
    mode='markers',
    marker=dict(color='black', size=5),  # Pontos pretos
    name='Temperatura'
))

fig_global.update_layout(
    title="📈 Tendência Global da Mudança de Temperatura (1961-2022)",
    xaxis_title="Ano",
    yaxis_title="Mudança Média de Temperatura (°C)",
    template="plotly_white"
)

fig_global.show()


fig_global = px.line(global_temp_trend, x="Year", y="Temperature_Change",
                     title="📈 Tendência Global da Mudança de Temperatura (1961-2022)",
                     labels={"Temperature_Change": "Mudança Média de Temperatura (°C)", "Year": "Ano"},
                     markers=True)

fig_global.update_traces(line=dict(color="red", width=3))
fig_global.show()

In [63]:
top_countries = df_melted.groupby('Country')['Temperature_Change'].mean().nlargest(10).index
df_top_countries = df_melted[df_melted['Country'].isin(top_countries)]
# df_top_countries = df_top_countries.sort_values(by=['Temperature_Change'], ascending=True)

# Comparando os países mais afetados
fig_countries = px.line(df_top_countries, x="Year", y="Temperature_Change", color="Country",
                        title="🌍 Comparação da Mudança de Temperatura para os 10 Países Mais Afetados",
                        labels={"Temperature_Change": "Mudança Média de Temperatura (°C)", "Year": "Ano"},
                        markers=True)

fig_countries.update_traces(line=dict(width=2))
fig_countries.show()